In [1]:
import tushare as ts
import pandas as pd
from influxdb import DataFrameClient
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [4]:
data = []
for date in tqdm(pd.date_range(start='2018-01-01', end='2020-05-14')):
    date_str = date.strftime("%Y%m%d")
    df = pro.daily(trade_date=date_str)
    data.append(df)

100%|██████████| 865/865 [06:04<00:00,  2.37it/s]


In [5]:
stock_df = pd.concat(data)

In [9]:
len(stock_df)

3979230

In [6]:
companys = set(stock_df['ts_code'])

In [7]:
len(companys)

3869

In [11]:
stock_by_company = []
for company_id in tqdm(companys):
    df = stock_df[stock_df['ts_code']==company_id]
    stock_by_company.append((df, company_id))

100%|██████████| 3869/3869 [14:36<00:00,  4.41it/s]


In [8]:
client = DataFrameClient('localhost', 8086, database='stock', username='influx', password='2020liufeng')

In [17]:
for i in tqdm(range(len(companys))):
    df, companyId = stock_by_company[i]
    df = df.drop(columns=['ts_code'])
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df = df.set_index('trade_date')
    client.write_points(df, 'stock', protocol="line", tags={'companyId': companyId})

41%|████▏     | 1605/3869 [14:04<19:51,  1.90it/s]


InfluxDBServerError: {"error":"timeout"}

In [9]:
res = client.query("select * from stock where companyId='000001.SZ'")

In [10]:
res['stock']

,amount,change,close,companyId,high,low,open,pct_chg,pre_close,vol
2015-01-05 00:00:00+00:00,4.565388e+06,0.18,16.02,000001.SZ,16.28,15.60,15.99,1.1400,15.84,2860436.43
2015-01-06 00:00:00+00:00,3.453446e+06,-0.24,15.78,000001.SZ,16.39,15.55,15.85,-1.5000,16.02,2166421.40
2015-01-07 00:00:00+00:00,2.634796e+06,-0.30,15.48,000001.SZ,15.83,15.30,15.56,-1.9000,15.78,1700120.67
2015-01-08 00:00:00+00:00,2.128003e+06,-0.52,14.96,000001.SZ,15.57,14.90,15.50,-3.3600,15.48,1407714.21
2015-01-09 00:00:00+00:00,3.835378e+06,0.12,15.08,000001.SZ,15.87,14.71,14.90,0.8000,14.96,2508500.23
...,...,...,...,...,...,...,...,...,...,...
2020-05-08 00:00:00+00:00,1.297925e+06,0.26,13.95,000001.SZ,14.02,13.68,13.76,1.8992,13.69,934781.70
2020-05-11 00:00:00+00:00,8.591566e+05,0.05,14.00,000001.SZ,14.13,13.90,13.92,0.3584,13.95,612862.29
2020-05-12 00:00:00+00:00,7.721095e+05,-0.21,13.79,000001.SZ,14.05,13.72,13.95,-1.5000,14.00,558511.14
2020-05-13 00:00:00+00:00,8.710620e+05,-0.16,13.63,000001.SZ,13.78,13.53,13.75,-1.1603,13.79,640358.79
